## 1-Packages
Import dependences


In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

## We'll Skip this for now

Define class Matrix Factorization

In [11]:
# class MF(object):
#     """docstring for CF"""
#     def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
#             learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
#         self.Y_raw_data = Y_data
#         self.K = K
#         # regularization parameter
#         self.lam = lam
#         # learning rate for gradient descent
#         self.learning_rate = learning_rate
#         # maximum number of iterations
#         self.max_iter = max_iter
#         # print results after print_every iterations
#         self.print_every = print_every
#         # user-based or item-based
#         self.user_based = user_based
#         # number of users, items, and ratings. Remember to add 1 since id starts from 0
#         self.n_users = int(np.max(Y_data[:, 0])) + 1 
#         self.n_items = int(np.max(Y_data[:, 1])) + 1
#         self.n_ratings = Y_data.shape[0]
        
#         if Xinit is None: # new
#             self.X = np.random.randn(self.n_items, K)
#         else: # or from saved data
#             self.X = Xinit 
        
#         if Winit is None: 
#             self.W = np.random.randn(K, self.n_users)
#         else: # from daved data
#             self.W = Winit
            
#         # normalized data, update later in normalized_Y function
#         self.Y_data_n = self.Y_raw_data.copy()

# # Function to normalize utility matrix
#     def normalize_Y(self):
#         if self.user_based:
#             user_col = 0
#             item_col = 1
#             n_objects = self.n_users

#         # if we want to normalize based on item, just switch first two columns of data
#         else: # item bas
#             user_col = 1
#             item_col = 0 
#             n_objects = self.n_items

#         users = self.Y_raw_data[:, user_col] 
#         self.mu = np.zeros((n_objects,))
#         for n in range(n_objects):
#             # row indices of rating done by user n
#             # since indices need to be integers, we need to convert
#             ids = np.where(users == n)[0].astype(np.int32)
#             # indices of all ratings associated with user n
#             item_ids = self.Y_data_n[ids, item_col] 
#             # and the corresponding ratings 
#             ratings = self.Y_data_n[ids, 2]
#             # take mean
#             m = np.mean(ratings) 
#             if np.isnan(m):
#                 m = 0 # to avoid empty array and nan value
#             self.mu[n] = m
#             # normalize
#             self.Y_data_n[ids, 2] = ratings - self.mu[n]

# # Loss Function
#     def loss(self):
#         L = 0 
#         for i in range(self.n_ratings):
#             # user, item, rating
#             n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
#             L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
#         # take average
#         L /= self.n_ratings
#         # regularization, don't ever forget this 
#         L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
#         return L 


# # Get items rated, and users have rated
#     def get_items_rated_by_user(self, user_id):
#         """
#         get all items which are rated by user user_id, and the corresponding ratings
#         """
#         ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
#         item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
#         ratings = self.Y_data_n[ids, 2]
#         return (item_ids, ratings)
        
        
#     def get_users_who_rate_item(self, item_id):
#         """
#         get all users who rated item item_id and get the corresponding ratings
#         """
#         ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
#         user_ids = self.Y_data_n[ids, 0].astype(np.int32)
#         ratings = self.Y_data_n[ids, 2]
#         return (user_ids, ratings)
    

# # Update Parameters
#     def updateX(self):
#         for m in range(self.n_items):
#             user_ids, ratings = self.get_users_who_rate_item(m)
#             Wm = self.W[:, user_ids]
#             # gradient
#             grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
#                                                self.lam*self.X[m, :]
#             self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
#     def updateW(self):
#         for n in range(self.n_users):
#             item_ids, ratings = self.get_items_rated_by_user(n)
#             Xn = self.X[item_ids, :]
#             # gradient
#             grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
#                         self.lam*self.W[:, n]
#             self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))

# # Run Model
#     def fit(self):
#         self.normalize_Y()
#         for it in range(self.max_iter):
#             self.updateX()
#             self.updateW()
#             if (it + 1) % self.print_every == 0:
#                 rmse_train = self.evaluate_RMSE(self.Y_raw_data)
#                 print('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)

# # Make predictions
#     def pred(self, u, i):
#         """ 
#         predict the rating of user u for item i 
#         if you need the un
#         """
#         u = int(u)
#         i = int(i)
#         if self.user_based:
#             bias = self.mu[u]
#         else: 
#             bias = self.mu[i]
#         pred = self.X[i, :].dot(self.W[:, u]) + bias 
#         # truncate if results are out of range [0, 5]
#         if pred < 0:
#             return 0 
#         if pred > 5: 
#             return 5 
#         return pred 
        
    
#     def pred_for_user(self, user_id):
#         """
#         predict ratings one user give all unrated items
#         """
#         ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
#         items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
#         y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
#         predicted_ratings= []
#         for i in range(self.n_items):
#             if i not in items_rated_by_u:
#                 predicted_ratings.append((i, y_pred[i]))
        
#         return predicted_ratings
    
# # Root Mean Square Error
#     def evaluate_RMSE(self, rate_test):
#         n_tests = rate_test.shape[0]
#         SE = 0 # squared error
#         for n in range(n_tests):
#             pred = self.pred(rate_test[n, 0], rate_test[n, 1])
#             SE += (pred - rate_test[n, 2])**2 

#         RMSE = np.sqrt(SE/n_tests)
#         return RMSE

### Test on MovieLens100k Dataset

In [12]:
# r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

# ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
# ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

# rate_train = ratings_base.values
# rate_test = ratings_test.values

# # indices start from 0
# rate_train[:, :2] -= 1
# rate_test[:, :2] -= 1

##### First 5 elements in rate_train

In [13]:
# rate_train[:5]

##### Normalize based on users

In [14]:
# rs = MF(rate_train, K = 10, lam = .1, print_every = 10, 
#     learning_rate = 0.75, max_iter = 100, user_based = 1)
# rs.fit()
# # evaluate on test data
# RMSE = rs.evaluate_RMSE(rate_test)
# print('\nUser-based MF, RMSE =', RMSE)

##### Normalize based on items

In [15]:
# rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
# rs.fit()
# # evaluate on test data
# RMSE = rs.evaluate_RMSE(rate_test)
# print('\nItem-based MF, RMSE =', RMSE)

##### Without regularization to see how worse it is

In [16]:
# rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
# rs.fit()
# # evaluate on test data
# RMSE = rs.evaluate_RMSE(rate_test)
# print('\nItem-based MF, RMSE =', RMSE)

### Now let's try with SVD

In [17]:
class SVDMatrixFactorization:
    """
    Matrix Factorization using Singular Value Decomposition (SVD)
    Handles both user-based and item-based recommendation systems
    """
    def __init__(self, Y_data, K, user_based=1):
        """
        Initialize SVD Matrix Factorization
        
        Parameters:
        -----------
        Y_data : numpy array
            Rating data with columns [user_id, item_id, rating]
        K : int
            Number of latent factors
        user_based : bool, optional (default=1)
            Whether to use user-based or item-based approach
        """
        self.Y_raw_data = Y_data
        self.K = K  # Number of latent factors
        self.user_based = user_based
        # Determine number of users and items
        self.n_users = int(np.max(Y_data[:, 0])) + 1
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        # Prepare rating matrix
        self.rating_matrix = self._create_rating_matrix()
        
        # Normalized rating matrix and mean ratings
        self.normalized_matrix, self.user_means = self._normalize_ratings()
        
    def _create_rating_matrix(self):
        """
        Create a sparse rating matrix from raw data
        
        Returns:
        --------
        numpy array
            Rating matrix with users as rows and items as columns
        """
        # Initialize rating matrix with zeros
        rating_matrix = np.zeros((self.n_users, self.n_items))
        
        # Fill matrix with ratings
        for user, item, rating in self.Y_raw_data:
            rating_matrix[int(user), int(item)] = rating
        
        return rating_matrix
    
    def _normalize_ratings(self):
        """
        Normalize ratings by subtracting user means
        
        Returns:
        --------
        tuple: (normalized_matrix, user_means)
        """
        # Calculate mean ratings for each user
        user_means = np.zeros(self.n_users)
        normalized_matrix = self.rating_matrix.copy()
        
        for user in range(self.n_users):
            # Get user's rated items
            rated_items = normalized_matrix[user, :] > 0
            
            if np.sum(rated_items) > 0:
                # Calculate mean of rated items
                user_means[user] = np.mean(normalized_matrix[user, rated_items])
                
                # Subtract mean from rated items
                normalized_matrix[user, rated_items] -= user_means[user]
        
        return normalized_matrix, user_means
    
    def fit(self):
        """
        Perform SVD decomposition on normalized rating matrix
        
        Returns:
        --------
        self: Fitted model with decomposed matrices
        """
        # Perform SVD on normalized matrix
        U, sigma, VT = np.linalg.svd(self.normalized_matrix, full_matrices=False)
        
        # Truncate to K latent factors
        U_reduced = U[:, :self.K]
        sigma_reduced = np.diag(sigma[:self.K])
        VT_reduced = VT[:self.K, :]
        
        # Store decomposed matrices
        self.U = U_reduced
        self.sigma = sigma_reduced
        self.VT = VT_reduced
        
        return self
    
    def pred(self, user, item):
        """
        Predict rating for a specific user-item pair
        
        Parameters:
        -----------
        user : int
            User ID
        item : int
            Item ID
        
        Returns:
        --------
        float: Predicted rating
        """
        # Reconstruct rating using SVD components
        # Reconstruct with top K singular values
        reconstructed_rating = np.dot(
            np.dot(self.U[user, :], self.sigma), 
            self.VT[:, item]
        )
        
        # Add back user mean
        predicted_rating = reconstructed_rating + self.user_means[user]
        
        # Clip rating to valid range
        return np.clip(predicted_rating, 0, 5)
    
    def pred_for_user(self, user_id):
        """
        Predict ratings for all unrated items for a user
        
        Parameters:
        -----------
        user_id : int
            User ID to predict ratings for
        
        Returns:
        --------
        list: Predicted (item_id, rating) pairs
        """
        # Get items already rated by the user
        rated_items = np.where(self.rating_matrix[user_id, :] > 0)[0]
        
        # Predict ratings for unrated items
        predicted_ratings = []
        for item in range(self.n_items):
            if item not in rated_items:
                pred_rating = self.pred(user_id, item)
                predicted_ratings.append((item, pred_rating))
        
        return predicted_ratings

    def evaluate_RMSE(self, rate_test):
        """
        Calculate Root Mean Square Error on test data
        
        Parameters:
        -----------
        rate_test : numpy array
            Test ratings data
        
        Returns:
        --------
        float: RMSE value
        """
        n_tests = rate_test.shape[0]
        SE = 0  # Squared Error
        
        for n in range(n_tests):
            pred = self.pred(int(rate_test[n, 0]), int(rate_test[n, 1]))
            SE += (pred - rate_test[n, 2])**2 
        
        RMSE = np.sqrt(SE/n_tests)
        return RMSE

##### Let's run on MovieLens100k Dataset

In [18]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('../data/ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('../data/ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1
# Remove timnestamp for training proceess
rate_train = rate_train[:,:-1]
rate_test = rate_test[:,:-1]

In [19]:
# First five elements
rate_train[:5]

array([[0, 0, 5],
       [0, 1, 3],
       [0, 2, 4],
       [0, 3, 3],
       [0, 4, 3]])

In [20]:
movies_set_raw = pd.read_csv("../data/ml-100k/u.item", encoding="latin-1", sep="|", names=["id", "name", "col3", "col4", "col5", "col6", "col7", "col8", "col9", "col10", "col11", "col12", "col13", "col14", "col15", "col16", "col17", "col18", "col19", "col20", "col21", "col22", "col23", "col24"])

movies_set = movies_set_raw.iloc[:,:2]

In [21]:
# rs_svd = SVDMatrixFactorization(rate_train, K = 5, user_based=1)
# rs_svd.fit()
# # evaluate on test data
# RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
# print('\nMF with SVD, RMSE =', RMSE_svd)

In [22]:
# rs_svd = SVDMatrixFactorization(rate_train, K = 5, user_based=0)
# rs_svd.fit()
# # evaluate on test data
# RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
# print('\nMF with SVD, RMSE =', RMSE_svd)

##### Now let's recommend the top 10 must watch movies

1. for new user

In [23]:
rs_svd = SVDMatrixFactorization(rate_train, K = 500, user_based=0)
rs_svd.fit()
RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
print('\nMF with SVD, K = 500 , RMSE =', RMSE_svd)
rs_svd = SVDMatrixFactorization(rate_train, K = 100, user_based=0)
rs_svd.fit()
RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
print('\nMF with SVD, K = 100, RMSE =', RMSE_svd)
rs_svd = SVDMatrixFactorization(rate_train, K = 200, user_based=0)
rs_svd.fit()
RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
print('\nMF with SVD, K = 200, RMSE =', RMSE_svd)
rs_svd = SVDMatrixFactorization(rate_train, K = 1000, user_based=0)
rs_svd.fit()
RMSE_svd = rs_svd.evaluate_RMSE(rate_test)
print('\nMF with SVD, K = 1000, RMSE =', RMSE_svd)
# rcm = rs_svd.recommend_new_user(top_n=10)
# print("Top 10 Must Watch Movie:")
# idx = 1
# for i in rcm:
#     print(str(idx) + ". " + i)
#     idx += 1


MF with SVD, K = 500 , RMSE = 1.0594251736343696

MF with SVD, K = 100, RMSE = 1.033888535270703

MF with SVD, K = 200, RMSE = 1.048198727924585

MF with SVD, K = 1000, RMSE = 1.0603799056362448


##### Now let's run with the 2018 Dataset

In [24]:
# movies_set_2018_raw = pd.read_csv("ml-latest-small/movies.csv", encoding='latin-1')
# movies_set_2018 = movies_set_2018_raw.drop(['genres'],axis=1)
# movies_set_2018.rename(columns={'movieId': 'id', 'title':'name'},inplace=True)
# movies_set_2018

In [25]:
# latest_df = pd.read_csv("ml-latest-small/ratings.csv")
# latest_df_arr = latest_df.values[:,:-1]
# latest_df_arr

##### Seperate train and test set

In [26]:
# from sklearn.model_selection import train_test_split

# rate_train_2018, rate_test_2018 = train_test_split(latest_df_arr, test_size=0.33, random_state=42)
# print(rate_train_2018.shape, rate_test_2018.shape)

##### Fit model

In [27]:
# model_svd_2 = SVDMatrixFactorization(rate_train_2018, K=10, movies_set=movies_set_2018, user_based=0)
# model_svd_2.fit()

# RMSE_svd_2018 = model_svd_2.evaluate_RMSE(rate_test_2018)
# print('\nMF with SVD, RMSE =', RMSE_svd_2018)

# # rcm2 = model_svd_2.recommend_new_user(top_n=10)
# # print("Top 10 Must Watch Movie:")
# # idx = 1
# # for i in rcm2:
# #     print(str(idx) + ". " + i)
# #     idx += 1

In [28]:
# rcm2 = model_svd_2.rec_new_user(top_n=10)
# print("Top 10 Must Watch Movie:")
# # idx = 1
# # for i in rcm2:
# #     print(str(idx) + ". " + i)
# #     idx += 1
# rcm2